In [5]:
#!/usr/bin/env python3
import logging
import tarfile
import glob

from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

input_files = glob.glob('nums[0-9].txt')
all_files=[]

rc = ReplicaCatalog()
for file in input_files:
    all_files.append(File(file))
    rc.add_replica("local", File(file), str(Path(".").resolve() / file))
    
rc.write()

#Transformation
separate = Transformation( "separate.py",
        site = "local",
        pfn = "/home/scitech/shared-data/Workflow/Workflow3/separate.py",
        is_stageable=True)

count = Transformation("count.py",
        site = "local",
        pfn = str(Path(".").resolve() / "count.py"),
        is_stageable = True)

tar = Transformation("tar",
        site = "condorpool",
        pfn = "/bin/tar",
        is_stageable = False)

tc = TransformationCatalog()\
    .add_transformations(separate,count,tar)\
    .write()


#Workflow
wf = Workflow("Even_Odd", infer_dependencies=True)

even_nums = File("even.txt")
odd_nums = File("odd.txt")

job_separate = Job(separate)\
                    .add_inputs(*all_files)\
                    .add_outputs(even_nums,odd_nums)

even_count = File("even_count.txt")

job_even_count = Job(count)\
                    .add_args(even_nums,even_count)\
                    .add_inputs(even_nums)\
                    .add_outputs(even_count)

odd_count = File("odd_count.txt")

job_odd_count = Job(count)\
                    .add_args(odd_nums,odd_count)\
                    .add_inputs(odd_nums)\
                    .add_outputs(odd_count)

result = File("results.tar.gz")

job_compress = Job(tar)\
            .add_args("-cvzf",result, even_count, odd_count)\
            .add_inputs(even_count, odd_count)\
            .add_outputs(result)

wf.add_jobs(job_separate,job_even_count,job_odd_count,job_compress)

In [6]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.13 20:05:16.326 UTC:    
2020.07.13 20:05:16.333 UTC:   ----------------------------------------------------------------------- 
2020.07.13 20:05:16.338 UTC:   File for submitting this DAG to HTCondor           : Even_Odd-0.dag.condor.sub 
2020.07.13 20:05:16.344 UTC:   Log of DAGMan debugging messages                 : Even_Odd-0.dag.dagman.out 
2020.07.13 20:05:16.350 UTC:   Log of HTCondor library output                     : Eve

[##################################################] 100.0% ..Success (Completed: 19, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/Workflow/Workflow3/scitech/pegasus/Even_Odd/run0005
 Total jobs         :     19 (100.00%)
 # jobs succeeded   :     19 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total co